## Renames Resting State Eyetracking Files (.sh)

This script will read the edf to extract dates from the files itself and will determine the ID based on the Participant ID folder it is located in. This script should only be run after all the files have been sorted into their respective folders. The script will check if a session folder ( ID in lowercase if pulling from the source, e.g bayXXXX) is in the correct format before renaming it, along with the files within it. If dates cannot be extracted from the file directly or the session folder includes unexpected characters, it will prompt the user to enter the information manually.

*Note that the script determines ID based on the folders instead of the files because any nanming errors are only corrected for the folder (and not the files inside) post data collection. 

In [ ]:
#!/bin/bash

renamed_folders_count=0

extract_date() {
    local file="$1"
    local date_info=$(strings "$file" | grep "^DATE:" | awk -F "DATE:" '{print $2}')
    local month=$(echo "$date_info" | awk '{print $2}')
    local day=$(echo "$date_info" | awk '{print $3}')
    local year=$(echo "$date_info" | awk '{print $NF}')

    local month_num=$(date -d "$month 1 2000" +%m 2> /dev/null | sed 's/^0*//')

    if [[ -z $month_num ]]; then
        echo "Error: Unable to extract date from $file."
        return 1
    else
        printf -v date_str "%4d%02d%02d" "$year" "$month_num" "$day"
        echo "$date_str"
    fi
}

rename_file() {
    local file="$1"
    local session_dir="$2"
    local id="$3"
    local date="$4"
    local new_name

    case "$file" in
        *.edf)
            new_name="${session_dir}/sub-${id}_${date}_resting_fixation.edf"
            ;;
        *actual_Trial*.dat)
            new_name="${session_dir}/sub-${id}_${date}_resting_trial_fixation.dat"
            ;;
        *actual_firsttrialFixation*.dat)
            new_name="${session_dir}/sub-${id}_${date}_resting_first_fixation.dat"
            ;;
    esac

    if [[ -n $new_name ]]; then
        mv "$file" "$new_name"
    fi
}

for id_folder in */; do
    if [[ -d "$id_folder" ]]; then
        id=${id_folder%/}
        for session_folder in "${id_folder}"*/; do
            session_name=$(basename "$session_folder")
            date=""

            if ! [[ $session_name =~ ^[a-zA-Z0-9]+$ ]]; then
                echo "Folder '$session_folder' has an inconsistent naming format."
                echo -n "Enter the correct folder name (should only contain alphanumeric characters) for '$session_folder': "
                read id
                while ! [[ $id =~ ^[a-zA-Z0-9]+$ ]]; do
                    echo "Invalid ID format. Please enter a valid ID for '$session_folder': "
                    read id
                done
            fi

            if ! [[ $session_name =~ ^[0-9]{8}$ ]]; then
                while IFS= read -r -d '' file; do
                    if [[ -z $date ]]; then
                        extracted_date=$(extract_date "$file")
                        if [[ $? -eq 0 ]]; then
                            date=$extracted_date
                        fi
                    fi
                    rename_file "$file" "$session_folder" "$id" "$date"
                done < <(find "$session_folder" -maxdepth 1 -type f \( -name "*.edf" -o -name "actual_Trial*.dat" -o -name "actual_firsttrialFixation*.dat" \) -print0)
                
                if [[ -z $date ]]; then
                    echo "Error: Cannot find date for files in directory '$session_folder'"
                    echo -n "Enter the date for files in '$session_folder' in YYYYMMDD format (e.g. 20231211): "
                    read date
                    while ! [[ $date =~ ^[0-9]{8}$ ]]; do
                        echo "Invalid date format. Please enter a valid date for '$session_folder': "
                        read date
                    done
                fi

                new_session_folder="${id_folder}${date}"
                if [[ ! -d "$new_session_folder" ]]; then
                    mv "$session_folder" "$new_session_folder" && {
                        ((renamed_folders_count++))
                        echo "$session_folder -> $new_session_folder"
                    }
                fi
            fi
        done
    fi
done

echo "Renaming completed."
echo "Total session folders renamed: $renamed_folders_count"


Older Version

In [ ]:
#!/bin/bash

renamed_folders_count=0

extract_date() {
    local file="$1"
    local date_info=$(strings "$file" | grep "^DATE:" | awk -F "DATE:" '{print $2}')
    local month=$(echo "$date_info" | awk '{print $2}')
    local day=$(echo "$date_info" | awk '{print $3}')
    local year=$(echo "$date_info" | awk '{print $NF}')

    local month_num=$(date -d "$month 1 2000" +%m 2> /dev/null | sed 's/^0*//')

    if [[ -z $month_num ]]; then
        echo "Error: Unable to extract date from $file."
        return 1
    else
        printf -v date_str "%4d%02d%02d" "$year" "$month_num" "$day"
        echo "$date_str"
    fi
}

rename_files() {
    local session_dir="$1"
    local id="$2"
    local date="$3"

    local edf_file=$(find "$session_dir" -maxdepth 1 -name "*.edf")
    if [[ -n $edf_file ]]; then
        mv "$edf_file" "${session_dir}/sub-${id}_${date}_resting_fixation.edf"
    fi

    local trial_dat_file=$(find "$session_dir" -maxdepth 1 -name "actual_Trial*.dat")
    if [[ -n $trial_dat_file ]]; then
        mv "$trial_dat_file" "${session_dir}/sub-${id}_${date}_resting_trial_fixation.dat"
    fi

    local first_trial_dat_file=$(find "$session_dir" -maxdepth 1 -name "actual_firsttrialFixation*.dat")
    if [[ -n $first_trial_dat_file ]]; then
        mv "$first_trial_dat_file" "${session_dir}/sub-${id}_${date}_resting_first_fixation.dat"
    fi
}

for id_folder in */; do
    if [[ -d "$id_folder" ]]; then
        id=${id_folder%/}
        for session_folder in "${id_folder}"*/; do
            session_name=$(basename "$session_folder")
            date=""

            if ! [[ $session_name =~ ^[a-zA-Z0-9]+$ ]]; then
                echo "Folder '$session_folder' has an inconsistent naming format."
                echo -n "Enter the correct folder name (should only contain alphanumeric characters) for '$session_folder': "
                read id
                while ! [[ $id =~ ^[a-zA-Z0-9]+$ ]]; do
                    echo "Invalid ID format. Please enter a valid ID for '$session_folder': "
                    read id
                done
            fi

            if ! [[ $session_name =~ ^[0-9]{8}$ ]]; then
                for edf_file in "${session_folder}"*.edf; do
                    if [[ -f "$edf_file" ]]; then
                        extracted_date=$(extract_date "$edf_file")
                        if [[ $? -eq 0 ]]; then
                            date=$extracted_date
                            break
                        fi
                    fi
                done

                if [[ -z $date ]]; then
                    echo "Error: Cannot find date for files in directory '$session_folder'"
                    echo -n "Enter the date for files in '$session_folder' in DDMMYYYY format (e.g. 11122023): "
                    read date
                    while ! [[ $date =~ ^[0-9]{8}$ ]]; do
                        echo "Invalid date format. Please enter a valid date for '$session_folder': "
                        read date
                    done
                fi

                new_session_folder="${id_folder}${date}"
                if [[ ! -d "$new_session_folder" ]]; then
                    mv "$session_folder" "$new_session_folder" && {
                        ((renamed_folders_count++))
                        echo "$session_folder -> $new_session_folder"
                        rename_files "$new_session_folder" "$id" "$date"
                    }
                fi
            fi
        done
    fi
done

echo "Renaming completed."
echo "Total session folders renamed: $renamed_folders_count"
